In [4]:
import sys; sys.path.append('../')
from fit.datamodules.tomo_rec import Kanji_TRecFITDM
from fit.utils.tomo_utils import get_polar_rfft_coords_2D, get_polar_rfft_coords_sinogram
from fit.utils import denormalize, convert2DFT
from fit.modules import TRecTransformerModule

from matplotlib import pyplot as plt

import torch

import numpy as np

from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import ModelCheckpoint

import wget
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
from os.path import exists

In [5]:
seed_everything(22122020)

Seed set to 22122020


22122020

In [7]:
dm = Kanji_TRecFITDM(root_dir='./datamodules/data/Kanji', batch_size=8, num_angles=33)
# FIT: TRec + FBP vs FIT: TRec
with_fbp = True

dm.prepare_data()
dm.setup()

NotImplementedError: Subclasses of Dataset should implement __getitem__.

In [ ]:
angles = dm.gt_ds.get_ray_trafo().geometry.angles
det_len = dm.gt_ds.get_ray_trafo().geometry.detector.shape[0]

In [ ]:
img_shape = dm.gt_shape

In [ ]:
proj_r, proj_phi, src_flatten = get_polar_rfft_coords_sinogram(angles=angles, 
                                                               det_len=det_len)
target_r, target_phi, dst_flatten, order = get_polar_rfft_coords_2D(img_shape=img_shape)

In [ ]:
n_heads = 8
d_query = 32
model = TRecTransformerModule(d_model=n_heads * d_query, 
                              sinogram_coords=(proj_r, proj_phi),
                              target_coords=(target_r, target_phi),
                              src_flatten_coords=src_flatten, 
                              dst_flatten_coords=dst_flatten, 
                              dst_order=order,
                              angles=angles, 
                              img_shape=img_shape,
                              detector_len=det_len,
                              loss='prod', 
                              use_fbp=with_fbp, 
                              init_bin_factor=1, 
                              bin_factor_cd=5,
                              lr=0.0001, 
                              weight_decay=0.01, 
                              attention_type='linear', 
                              n_layers=4,
                              n_heads=n_heads, 
                              d_query=d_query, 
                              dropout=0.1, 
                              attention_dropout=0.1)

In [ ]:
trainer = Trainer(max_epochs=120, 
                  gpus=1,
                  callbacks=ModelCheckpoint(
                                            dirpath=None,
                                            save_top_k=1,
                                            verbose=False,
                                            save_last=True,
                                            monitor='Train/avg_val_mse',
                                            mode='min'
                                        ), 
                  deterministic=True)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
# Uncomment the next line if you want to train your own model. 
# trainer.fit(model, datamodule=dm);

In [ ]:
if not exists('./models/trec_kanji/kanji_trec.ckpt'):
    wget.download('https://download.fht.org/jug/fit/trec_model_kanji.ckpt',
                  out='./models/trec_kanji/kanji_trec.ckpt')
    
if not exists('./models/trec_kanji/kanji_trec_fbp.ckpt'):
    wget.download('https://download.fht.org/jug/fit/trec_fbp_model_kanji.ckpt',
                  out='./models/trec_kanji/kanji_trec_fbp.ckpt')

In [ ]:
if with_fbp:
    path = './models/trec_kanji/kanji_trec_fbp.ckpt'
else:
    path = './models/trec_kanji/kanji_trec.ckpt'

In [ ]:
model = TRecTransformerModule.load_from_checkpoint(path, 
                                               sinogram_coords=(proj_r, proj_phi),
                              target_coords=(target_r, target_phi),
                              src_flatten_coords=src_flatten, 
                              dst_flatten_coords=dst_flatten, 
                              dst_order=order, angles=angles, strict=False)

In [ ]:
test_res = trainer.test(model, datamodule=dm)

/home/tibuch/Apps/miniconda3/envs/fit/lib/python3.7/site-packages/pytorch_lightning/core/datamodule.py:424: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  f"DataModule.{name} has already been called, so it will not be called again. "
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/tibuch/Apps/miniconda3/envs/fit/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:106: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Testing: 0it [00:00, ?it/s]

bin_factor set to 1.


/home/tibuch/Apps/miniconda3/envs/fit/lib/python3.7/site-packages/torch/nn/functional.py:1794: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


30.720855712890625
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'Mean PSNR': 30.720855712890625, 'SEM PSNR': 0.031072383746504784}
--------------------------------------------------------------------------------


In [ ]:
for x_fc, fbp_fc, y_fc, y_real, (amp_min, amp_max) in dm.test_dataloader():
    break

In [ ]:
model.cpu();

In [ ]:
pred_img, pred_img_before_conv = model.get_imgs(x_fc, fbp_fc, y_fc, amp_min, amp_max)

/home/tibuch/Gitrepos/FourierImageTransformer/fit/modules/TRecTransformerModule.py:358: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448224956/work/aten/src/ATen/native/Copy.cpp:240.)
  pred_fc_[:, :tmp.shape[1]] = tmp


In [ ]:
# Before the projection we normalized the image, now we undo this for the visualization.
# After denormalization we set all pixels outside of the projection-area to zero
pred_img = denormalize(pred_img, dm.mean, dm.std) * dm.__get_circle__()
y_real = denormalize(y_real, dm.mean, dm.std) * dm.__get_circle__()

In [ ]:
dft_fbp = convert2DFT(x=fbp_fc[:,model.dst_flatten_order], amp_min=amp_min, amp_max=amp_max,
                              dst_flatten_order=model.dst_flatten_order, img_shape=model.hparams.img_shape)
fbp_img = torch.roll(torch.fft.irfftn(model.mask * dft_fbp[0], s=2 * (model.hparams.img_shape,)),
                     2 * (model.hparams.img_shape // 2,), (0, 1))

fbp_img = (fbp_img - fbp_img.min())*255/(fbp_img.max() - fbp_img.min())
fbp_img = fbp_img * dm.__get_circle__()

In [2]:
plt.figure(figsize=(15,5))
plt.subplot(1,3,1)
plt.imshow(fbp_img, cmap='gray', vmin=y_real[0].min(), vmax=y_real[0].max())
plt.title('Filtered Backprojection');
plt.subplot(1,3,2)
plt.imshow(pred_img[0].detach(), cmap='gray', vmin=y_real[0].min(), vmax=y_real[0].max())
plt.title('Prediction');
plt.subplot(1,3,3)
plt.imshow(y_real[0], cmap='gray')
plt.title('Ground Truth');

NameError: name 'plt' is not defined